In [101]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister
import qiskit.circuit.library as qulib
from qiskit.qasm3 import dump, dumps, Exporter
import numpy as np
import random
from typing import List, Union
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qmg.utils import MoleculeQuantumStateGenerator, ConditionalWeightsGenerator
import pandas as pd
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

def get_token(file_path):
    with open(file_path) as f:
        data = f.read()
    token = data.strip()
    return token

my_token = get_token("./docs/ibmq_tokens.txt")
service = QiskitRuntimeService(channel="ibm_quantum", token=my_token)
 
backend = service.least_busy(simulator=False, operational=True)
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
print(backend)

num_heavy_atom = 9
num_parameters = int(8 + 3*(num_heavy_atom - 2) * (num_heavy_atom + 3) / 2)
qubit_register_dict = {"atom_1": QuantumRegister(2, name="atom_1"),
                       "atom_i": QuantumRegister(2, name="atom_i")}
for i in range(1, num_heavy_atom):
    qubit_register_dict.update({f"bond_{i}": QuantumRegister(2, name=f"bond_{i}")})

clbit_register_dict = {}
for i in range(1, num_heavy_atom+1):
    clbit_register_dict.update({f"atom_{i}_m": ClassicalRegister(2, name=f"atom_{i}_m")})
    for j in range(1, i):
        clbit_register_dict.update({f"bond_{i}_{j}_m": ClassicalRegister(2, name=f"bond_{i}_{j}_m")})

<IBMBackend('ibm_sherbrooke')>


In [87]:
# get weight vector
trial_df = pd.read_csv("results_chemistry_constraint_bo/unconditional_9_4.csv")
index = 501

cwg = ConditionalWeightsGenerator(num_heavy_atom=num_heavy_atom, smarts=None, disable_connectivity_position=None)
random_weight_vector = np.zeros(cwg.length_all_weight_vector)
inputs = random_weight_vector
number_flexible_parameters = len(random_weight_vector[cwg.parameters_indicator == 0.])
partial_inputs = np.array(trial_df[trial_df["trial_index"] == index][[f"x{i+1}" for i in range(number_flexible_parameters)]])[0]
inputs[cwg.parameters_indicator == 0.] = partial_inputs
inputs = cwg.apply_chemistry_constraint(inputs)
weight_vector = inputs

In [88]:
weight_vector

array([0.        , 0.93771391, 0.33959516, 0.87208743, 0.        ,
       1.        , 0.        , 1.        , 0.92984925, 0.49968326,
       1.        , 0.99330715, 0.00669285, 0.        , 0.5       ,
       0.        , 1.        , 0.55639786, 1.        , 0.        ,
       0.95335631, 0.03454501, 0.01209868, 0.        , 1.        ,
       0.47865948, 0.5       , 0.        , 1.        , 0.61658179,
       0.99936443, 0.46670468, 0.01108171, 0.00579138, 0.12360936,
       0.85951755, 0.34363469, 0.52658671, 0.5       , 1.        ,
       0.        , 0.93303569, 0.        , 1.        , 0.40874672,
       1.        , 0.        , 0.49225377, 0.49225377, 0.00752682,
       0.00331678, 0.00464886, 0.        , 0.5       , 0.        ,
       0.78951158, 0.        , 1.        , 0.23592222, 1.        ,
       0.43559468, 1.        , 0.36484118, 0.91858667, 0.04533939,
       0.00331186, 0.49152386, 0.49152386, 0.00701669, 0.00331186,
       0.00331186, 0.40438467, 0.62031226, 0.        , 0.92181

In [89]:
list(qubit_register_dict.values())
circuit = QuantumCircuit(*list(qubit_register_dict.values()), *list(clbit_register_dict.values()))

def controlled_ry(control:int, target:int, digit:float):
    circuit.cry(np.pi*digit, control, target)

def reset_q_register(circuit, q_register, measures):
    """Reset the controlling qubits if they are in |1>."""
    with circuit.if_test((measures[0], True)):
        circuit.x(q_register[0])
    with circuit.if_test((measures[1], True)):
        circuit.x(q_register[1])

def reset_block(circuit, heavy_atom_idx):
    reset_q_register(circuit, qubit_register_dict["atom_i"],  clbit_register_dict["atom_2_m"])
    for k in range(1, heavy_atom_idx):
        reset_q_register(circuit, qubit_register_dict[f"bond_{k}"],  clbit_register_dict[f"bond_{heavy_atom_idx}_{k}_m"])

def measure_bond_block(circuit, heavy_atom_idx):
    for k in range(1, heavy_atom_idx):
        circuit.measure(qubit_register_dict[f"bond_{k}"],  clbit_register_dict[f"bond_{heavy_atom_idx}_{k}_m"])

# first two atoms part
circuit.ry(np.pi * weight_vector[0], 0)
circuit.x(1)
circuit.ry(np.pi * weight_vector[2], 2)
circuit.ry(np.pi * weight_vector[4], 3)
circuit.cx(0, 1)
controlled_ry(1, 2, weight_vector[3])
circuit.cx(2, 3)
controlled_ry(0, 1, weight_vector[1])
circuit.cx(1, 2)
controlled_ry(2, 3, weight_vector[5])

circuit.measure(qubit_register_dict["atom_1"], clbit_register_dict["atom_1_m"])
circuit.measure(qubit_register_dict["atom_i"], clbit_register_dict["atom_2_m"])
with circuit.if_test((clbit_register_dict["atom_2_m"], 0)) as else_:
    pass
with else_:
    circuit.ry(np.pi * weight_vector[6], 4)
    circuit.x(5)
    circuit.cx(4,5)
    controlled_ry(4, 5, weight_vector[7])
circuit.measure(qubit_register_dict["bond_1"], clbit_register_dict["bond_2_1_m"])

used_part = 8
heavy_atom_idx = 3
# Third atom and recrusive part
for heavy_atom_idx in range(3, num_heavy_atom+1):
    # reset qubits for qubit reuse
    reset_block(circuit, heavy_atom_idx - 1) # heavy atom idx starts with 2
    with circuit.if_test((clbit_register_dict[f"atom_{heavy_atom_idx-1}_m"], 0)) as else_:
        pass
    with else_:
        circuit.ry(np.pi * weight_vector[used_part], qubit_register_dict["atom_i"][0])
        circuit.ry(np.pi * weight_vector[used_part+1], qubit_register_dict["atom_i"][1])
        controlled_ry(qubit_register_dict["atom_i"][0], qubit_register_dict["atom_i"][1], weight_vector[used_part+2])
    circuit.measure(qubit_register_dict["atom_i"], clbit_register_dict[f"atom_{heavy_atom_idx}_m"])
    with circuit.if_test((clbit_register_dict[f"atom_{heavy_atom_idx}_m"], 0)) as else_:
        pass
    with else_:
        num_fixed = heavy_atom_idx-1
        num_flexible = 2*num_fixed
        bond_type_fixed_part = weight_vector[used_part+3: used_part+3+num_fixed]
        bond_type_flexible_part = weight_vector[used_part+3+num_fixed: used_part+3+num_fixed+num_flexible]
        for i in range(heavy_atom_idx-1):
            circuit.ry(np.pi * bond_type_fixed_part[i], qubit_register_dict[f"bond_{i+1}"][1])
            controlled_ry(qubit_register_dict[f"bond_{i+1}"][1], qubit_register_dict[f"bond_{i+1}"][0], bond_type_flexible_part[2*i]) # < 0.5
            controlled_ry(qubit_register_dict[f"bond_{i+1}"][0], qubit_register_dict[f"bond_{i+1}"][1], bond_type_flexible_part[2*i+1]) # > 0.5

    measure_bond_block(circuit, heavy_atom_idx)
    used_part += 3*heavy_atom_idx


In [90]:
num_sample = 5000

transpiled_qc = pm.run(circuit)
sampler = Sampler(mode=backend)
sampler.options.default_shots = num_sample
job = sampler.run([transpiled_qc])

print(f">>> Job ID: {job.job_id()}")

>>> Job ID: cygq8tk9b62g00822s70


In [91]:
from qiskit_ibm_runtime import QiskitRuntimeService

def get_token(file_path):
    with open(file_path) as f:
        data = f.read()
    token = data.strip()
    return token

service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='ibm-q/open/main',
    token=get_token("./docs/ibmq_tokens.txt")
)
job = service.job('cygq8tk9b62g00822s70')
job_result = job.result()

In [109]:
def post_process_results(job_result, num_heavy_atom):
    num_shots = job_result[0].data.atom_1_m.num_shots
    quantum_state_list = [""]*num_shots
    # node vector
    for i in range(1, num_heavy_atom+1):
        key = f"atom_{i}_m"
        partial_results = job_result[0].data[key].get_bitstrings()
        for z in range(num_shots):
            quantum_state_list[z] = quantum_state_list[z] + partial_results[z][::-1]
        
        # bond adjacency
        for j in range(1, i):
            key = f"bond_{i}_{j}_m"
            partial_results = job_result[0].data[key].get_bitstrings()
            for z in range(num_shots):
                quantum_state_list[z] = quantum_state_list[z] + partial_results[z][::-1]
                # remove bond disconnection
                if i - j == 1:
                    if (quantum_state_list[z][-2*j-2:-2*j] != "00") and (quantum_state_list[z][-2*j:] == "0"*(2*j)):
                        quantum_state_list[z] = quantum_state_list[z][:-1]+"1"
    return quantum_state_list

quantum_state_list = post_process_results(job_result, num_heavy_atom=9)
mg = MoleculeQuantumStateGenerator(heavy_atom_size=9)
quantum_state_list = [mg.post_process_quantum_state(qs, reverse=False) for qs in quantum_state_list]

In [112]:
from collections import Counter
smiles_list = []
for qs in quantum_state_list:
    smiles = mg.QuantumStateToSmiles(qs)
    if smiles:
        smiles_list.append(smiles)
smiles_dict = Counter(smiles_list)
print("Validity: ", len(smiles_list) / num_sample)
print("Uniqueness: ", len(smiles_dict) / num_sample)

Validity:  0.1078
Uniqueness:  0.0944


In [113]:
smiles_dict.keys()

dict_keys(['C', 'CC[C@@](C)(O)OC', 'CC[C@@H]1[C@@H](N)[C@@]1(O)O', 'CCN(O)[C@H](C)[C@@H](N)N', 'CO[C@]1(O)C[C@H]1C', 'C[C@]1(O)ON[C@@H]1CO', 'CC[C@@H](N)[C@@H](N)[C@@H](N)N', 'N[C@H](N(N)O)N1CCO1', 'C[C@H]1N[N@]2C[C@@]1(N)NN2', 'CN[C@H](NC)[C@H](N)CN', 'C[C@]1(N)C[C@@]1(O)O', 'CN1C[C@@H]([C@@H](O)O)C1', 'N=N[C@H](NN)NCN', 'CC[N@@]1N[C@]1(C)O', 'CC[C@H](NN)[C@@H](N)O', 'N[C@@H]1[C@@H](O)C[C@@]2(N)C[C@@H]12', 'CN[C@@H]1N[C@@H](OC)[C@@H]1C', 'CCNC[C@H]1C[C@H]1N', 'NN1[N@@]2C[N@@]3[N@@](N)[C@]13N2', 'C[C@H]1NN[C@H]1N(N)N', 'CC1=NN(C)[C@]1(C)N', 'NNC[C@H]1NC[C@H]1O', 'N', 'OOOO[C@H]1COO1', 'N[C@H]1CC(=O)N2C[N@]1C2', 'NN[C@]1([C@@H](O)ON)NO1', 'CCNNO[C@@H](C)ON', 'CN[C@]1(NC)C[N@@]1O', 'C1N[C@@]23[C@@H]4N=C([C@H]42)N13', 'CC[C@H](ON)OCN', 'N[C@@H](NNCO)C1=NC1', 'NC[C@]1(N)N[C@H]1NN', 'NC[C@]1([C@@H](O)O)CN1', 'CN(N)[C@@H]1NN(N)[C@H]1O', 'CNC[C@@H]1CN(C)N1', 'CO[C@@H]1O[C@]1(N)NN', 'CC[C@](N)(NN)N(C)N', 'C[C@@H](O)[C@@H](CO)NO', 'CN[C@H](CO)[C@H](C)N', 'CC[C@]12[C@@H]3[C@@H](N1CN)[N@@]32', 'C